In [1]:
import json
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('spider_data/train_spider.json') as f:
    query_data = json.load(f)

with open('spider_data/tables.json') as f:
    table_data = json.load(f)

multi_table_queries = [query for query in query_data if len(query['sql']['from']['table_units'])>1]

In [3]:
with open('spider_gold_mt.sql','w') as f:
    for rec in multi_table_queries[:500]:
        q = rec['query'].replace('\t',' ')
        line = f"{q}\t{rec['db_id']}\n"
        f.write(line)

In [24]:
with open('results_mt_v2.pkl','rb') as f:
    results_mt = pickle.load(f)

In [25]:
pred_queries = []
for query in results_mt[:500]:
    if query.lower().find('select')!=-1:
        end_index1 = query.find(';') if query.find(';')>0  else len(query)
        end_index2 = query.find('\n') if query.find('\n')>0 else len(query)
        
        filtered_query = query[query.lower().find('select'):min(end_index1,end_index2)]
        pred_queries.append(filtered_query)
    else:
        pred_queries.append(-1)

with open('pred_mt_v2.sql','w') as f:
    for rec in pred_queries:
        line = f"{rec}\n"
        f.write(line)

In [14]:
import pickle
with open('results_preprocessing_mt.pkl', 'rb') as f:
    res_codegen = pickle.load(f)

In [15]:
pred_codegen_queries = []
for query in res_codegen:
    if query.lower().find('select')!=-1:
        end_index1 = query.find(';') if query.find(';')>0  else len(query)
        end_index2 = query.find('\n') if query.find('\n')>0 else len(query)
        
        filtered_query = query[query.lower().find('select'):min(end_index1,end_index2)]
        pred_codegen_queries.append(filtered_query)
    else:
        pred_codegen_queries.append(-1)

with open('pred_codegen_mt.sql','w') as f:
    for rec in pred_codegen_queries:
        line = f"{rec}\n"
        f.write(line)

## unaliasing

In [29]:
import sqlglot
ua_queries = []
fails = 0
fails_list = []

for query in pred_codegen_queries:
    try:
        qt = sqlglot.parse_one(query)
        for t in qt.find_all(sqlglot.exp.Alias):
            t.replace(t.unalias())
        ua_queries.append(qt.sql())
    except:
        fails+=1
        fails_list.append(query)
        ua_queries.append(query)

Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argu

In [32]:
with open('pred_codegen_ua.sql','w') as f:
    for rec in ua_queries:
        line = f"{rec}\n"
        f.write(line)

## check parsability

In [35]:
import process_sql

In [43]:
fails = 0
for q, db in zip(pred_queries,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q)
    except:
        fails+=1

In [44]:
fails

3094

In [47]:
fails = 0
with open('pred_ua.sql') as f:
    qu = f.readlines()
    
for q, db in zip(qu,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q.strip())
    except:
        fails+=1

In [48]:
fails

1963

In [45]:
fails = 0
for q, db in zip(pred_codegen_queries,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q)
    except:
        fails+=1

In [46]:
fails

2853

In [49]:
fails = 0
with open('pred_codegen_ua.sql') as f:
    qu = f.readlines()
    
for q, db in zip(qu,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q.strip())
    except:
        fails+=1

In [50]:
fails

1832

## executability

In [26]:
with open('spider_gold_mt.sql') as f:
    gold_sql = f.readlines()

with open('pred_mt_v2.sql') as f:
    pred_queries = f.readlines()

with open('pred_codegen_mt.sql') as f:
    pred_codegen_queries = f.readlines()

In [27]:
len(gold_sql), len(pred_queries), len(pred_codegen_queries)

(500, 500, 500)

In [28]:
import sqlite3
execution_results = []
for g,p,pc in zip(gold_sql, pred_queries, pred_codegen_queries):
    results = {}
    g = g.strip().split('\t')
    p = str(p).strip()
    pc = str(pc).strip()

    db = f"spider_data/database/{g[1]}/{g[1]}.sqlite"
    con = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(g[0],con)
        results['gold_data'] = df
    except:
        results['gold_data'] = None

    try:
        df = pd.read_sql_query(p,con)
        results['pred_data'] = df
    except:
        results['pred_data'] = None

    try:
        df = pd.read_sql_query(pc,con)
        results['pred_codegen_data'] = df
    except:
        results['pred_codegen_data'] = None

    con.close()
    results['g'] = g
    results['p'] = p
    results['pc'] = pc

    execution_results.append(results)

In [29]:
gfails, pfails, pcfails = 0,0,0

for res in execution_results:
    if res['gold_data'] is None:
        gfails+=1

    if res['pred_data'] is None:
        pfails+=1

    if res['pred_codegen_data'] is None:
        pcfails+=1

In [30]:
gfails, pfails, pcfails

(0, 286, 294)

In [43]:
l = [rec for rec in execution_results if (rec['pred_data'] is None) and (rec['pred_codegen_data'] is not None)]

In [44]:
len(l)

52

In [34]:
l[10]

{'gold_data':    count(*)
 0       999,
 'pred_data': None,
 'pred_codegen_data':    COUNT(*)
 0      9959,
 'g': ['SELECT count(*) FROM trip AS T1 JOIN station AS T2 ON T1.end_station_id  =  T2.id WHERE T2.city != "San Francisco"',
  'bike_1'],
 'p': "SELECT COUNT(*) AS total_trips FROM trip WHERE end_station_name not ilike '%San%Francisco%'",
 'pc': "SELECT COUNT(*) FROM trip WHERE end_station_name != 'San Francisco'"}

## execution_accuracy

In [37]:
fails = 0
total_compare = 0
fails_list = []

for i, rec in enumerate(execution_results):
    if rec['gold_data'] is not None and rec['pred_data'] is not None:
        total_compare+=1

        if rec['gold_data'].shape!=rec['pred_data'].shape:
            fails+=1
            fails_list.append(i)
        else:
            if (np.sort(rec['pred_data'].astype('str').to_numpy().flatten())==np.sort(rec['gold_data'].astype('str').to_numpy().flatten())).all():
                pass
            else:
                fails+=1
                fails_list.append(i)

In [38]:
total_compare, fails, total_compare-fails

(214, 106, 108)

In [39]:
fails = 0
total_compare = 0
fails_list = []

for i, rec in enumerate(execution_results):
    if rec['gold_data'] is not None and rec['pred_codegen_data'] is not None:
        total_compare+=1

        if rec['gold_data'].shape!=rec['pred_codegen_data'].shape:
            fails+=1
            fails_list.append(i)
        else:
            if (np.sort(rec['pred_codegen_data'].astype('str').to_numpy().flatten())==np.sort(rec['gold_data'].astype('str').to_numpy().flatten())).all():
                pass
            else:
                
                fails+=1
                fails_list.append(i)

In [40]:
total_compare, fails, total_compare-fails

(206, 106, 100)

In [57]:
fails = 0
total_compare = 0
fails_list = []
corrected_by_cg = 0
corrected_list = []

for i, rec in enumerate(execution_results):
    fail_llm = True
    fail_codegen = True
    
    if rec['gold_data'] is not None and rec['pred_data'] is not None:
        total_compare+=1

        if rec['gold_data'].shape!=rec['pred_data'].shape:
            fails+=1
            fails_list.append(i)
            
        else:
            if (np.sort(rec['pred_data'].astype('str').to_numpy().flatten())==np.sort(rec['gold_data'].astype('str').to_numpy().flatten())).all():
                fail_llm=False
                pass
            else:
                fails+=1
                fails_list.append(i)

    if rec['gold_data'] is not None and rec['pred_codegen_data'] is not None:
        total_compare+=1

        if rec['gold_data'].shape!=rec['pred_codegen_data'].shape:
            fails+=1
            fails_list.append(i)
            
        else:
            if (np.sort(rec['gold_data'].astype('str').to_numpy().flatten())==np.sort(rec['pred_codegen_data'].astype('str').to_numpy().flatten())).all():
                fail_codegen=False
                pass
                
            else:
                fails+=1
                fails_list.append(i)

    if fail_llm==True and fail_codegen==False:
        corrected_by_cg+=1
        corrected_list.append(i)

In [58]:
corrected_by_cg

23

In [62]:
execution_results[corrected_list[15]]

{'gold_data':   Hardware_Model_name
 0  XPERIA T, XPERIA J,
 'pred_data':     Model_name
 0  X5 high end,
 'pred_codegen_data':   Hardware_Model_name
 0  XPERIA T, XPERIA J,
 'g': ['SELECT T2.Hardware_Model_name FROM chip_model AS T1 JOIN phone AS T2 ON T1.Model_name  =  T2.chip_model WHERE T1.Launch_year = 2002 ORDER BY T1.RAM_MiB DESC LIMIT 1;',
  'phone_1'],
 'p': 'SELECT chip_model.Model_name FROM chip_model WHERE launch_year = 2002 ORDER BY RAM_MiB DESC LIMIT 1',
 'pc': 'SELECT phone.hardware_model_name FROM phone JOIN chip_model ON phone.chip_model = chip_model.model_name WHERE chip_model.launch_year = 2002 ORDER BY chip_model.ram_mib DESC LIMIT 1'}